In [ ]:
# 这里输入API KEY
API_KEY = ""
OPENAI_BASE_URL='https://ggdl.haohaozhongzhuan.com/v1'

In [ ]:
def convert_pil_image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:image/png;base64,{img_str}"

In [ ]:
from openai import OpenAI

def chat(prompt, image1=None, image2=None, history=None,
         api_key=None,
         base_url=None):
    """
    prompt: str
    image: PIL.Image 或 None
    history: list[dict] 或 None
    api_key: str, OpenAI / vLLM 的 key
    base_url: str, 形如 https://xxx.com/v1
    """
    # 1. 构造 client
    client = OpenAI(
        api_key=API_KEY,
        base_url=OPENAI_BASE_URL,
    )

    # 2. 构造当前消息
    if image1 and image2:
        current_msg = {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {"url": convert_pil_image_to_base64(image1)}
                },
                {
                    "type": "image_url",
                    "image_url": {"url": convert_pil_image_to_base64(image2)}
                },
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    else:
        current_msg = {
            "role": "user",
            "content": [{"type": "text", "text": prompt}]
        }

    # 3. 拼历史
    history = history if history else []
    messages = history + [current_msg]

    # 4. 请求
    response = client.chat.completions.create(
        model="gemini-2.5-flash-image-preview",   # 按需修改
        messages=messages,
        temperature=0.0,
        max_tokens=2048,
    )

    response_text = response.choices[0].message.content
    messages.append({"role": "assistant", "content": response_text})
    return response_text, messages

In [ ]:
HINT_TEXT = ("""
把这张照片改成黑白图
""")

In [ ]:
import os
import json
from PIL import Image
from tqdm import tqdm
import chardet
import sys
import base64
from io import BytesIO

# img_path="D:\\benchmark\\datasets--likaixin--ScreenSpot-Pro\\snapshots\\f362949f57216063e6fe4b6e61b3e12340a351a8\\images\\android_studio_mac\\screenshot_2024-11-05_16-01-19.png"
img_path = "D:\\dev\\work\\python-script\\test.png"
image1 = Image.open(img_path).convert('RGB')
image2 = Image.open(img_path).convert('RGB')
history = []
position_inference_prompt = HINT_TEXT.format(instruction="the trash bin icon")
hint_text, history = chat(position_inference_prompt, image1,image2, history)

In [ ]:
hint_text